# Snapshot Fields Walkthrough

This notebook demonstrates how to use `snapshot_fields.py` and explains what each function does.


## Overview of functions

- `read_snapshot_particles`: read particle positions/velocities from a split HDF5 snapshot.
- `density_field`: mass-assign particles to a grid to build density and overdensity.
- `momentum_field`: build the momentum density field `q = (1+delta) * v_avg`.
- `velocity_field`: compute mass-weighted average velocity per grid cell.
- `matter_power_spectrum`: compute matter `P(k)` from the overdensity.
- `omega_power_spectrum`: compute curl power spectrum from the momentum field.
- `snapshot_products`: convenience wrapper that runs the full pipeline.
- `save_products_npy`: save outputs to `.npy` files.


In [1]:
import snapshot_fields as sf

# Optional: adjust these for your environment
BASE_PATH = '.'  # directory containing snapdir_###
SNAP_NUM = 0
NGRID = 1024
MAS = 'CIC'
OUT_DIR = 'outputs'


## 1) Read particle data

`read_snapshot_particles` loads particle arrays and snapshot metadata.
It returns a dict with `coordinates`, `velocities`, `header`, and `box_size`.


In [2]:
data = sf.read_snapshot_particles(BASE_PATH, SNAP_NUM)
pos = data['coordinates']
vel = data['velocities']
box_size = data['box_size']
pos.shape, vel.shape, box_size


TimeoutError: [Errno 60] Unable to synchronously open file (file read failed: time = Mon Dec 22 11:48:04 2025
, filename = 'snapdir_000/snap_000.7.hdf5', file descriptor = 88, errno = 60, error message = 'Operation timed out', buf = 0x16b9f36c0, total read size = 8, bytes this sub-read = 8, offset = 0)

## 2) Build density and overdensity

`density_field` returns `(rho, delta)` on the grid.
- `rho` is the raw mass-assigned density.
- `delta = rho / <rho> - 1`.


In [ ]:
rho, delta = sf.density_field(pos, NGRID, box_size, mas=MAS, verbose=True)
rho.shape, delta.min(), delta.max()


## 3) Build momentum and velocity fields

`momentum_field` returns `(qx, qy, qz, rho)` where q is the momentum density.
`velocity_field` returns `(vx, vy, vz, rho)` which is the mass-weighted average velocity.


In [ ]:
qx, qy, qz, _ = sf.momentum_field(pos, vel, NGRID, box_size, mas=MAS, rho=rho, verbose=True)
vx, vy, vz, _ = sf.velocity_field(pos, vel, NGRID, box_size, mas=MAS, rho=rho, verbose=True)
vx.shape, qx.shape


## 4) Matter power spectrum

`matter_power_spectrum` computes `P(k)` from the overdensity field.


In [ ]:
k_m, pk_m, _ = sf.matter_power_spectrum(delta, box_size, mas=MAS, threads=1, verbose=True)
k_m[:5], pk_m[:5]


## 5) Curl (omega) power spectrum

`omega_power_spectrum` computes the curl power spectrum of the momentum field.


In [ ]:
k_om, pk_om, nm_om = sf.omega_power_spectrum(qx, qy, qz, NGRID, mas=MAS, threads=1)
k_om[:5], pk_om[:5], nm_om[:5]


## 6) One-call pipeline + save outputs

`snapshot_products` runs the full pipeline and returns a dict.
`save_products_npy` writes everything to `.npy` files.


In [ ]:
products = sf.snapshot_products(
    base_path=BASE_PATH,
    snap_num=SNAP_NUM,
    ngrid=NGRID,
    box_size=box_size,
    mas=MAS,
    verbose=True,
)
sf.save_products_npy(products, OUT_DIR)
